#### Introduction to ML and OOD 

- Resources:
    - [Scikit-learn documentation](https://scikit-learn.org/stable/modules/outlier_detection.html) 

#### For the next experiments our OOD class will be for samples with DF class and the rest will be considered In-distribution

In [13]:
import pandas as pd

groundtruth = pd.read_csv("../data/ISIC2019/ISIC_2019_Training_GroundTruth.csv")
groundtruth = groundtruth.set_index("image")
oodist = list(groundtruth.index[groundtruth.DF == 1])
indist = list(groundtruth.index[groundtruth.DF != 1])

In [15]:
len(indist), len(oodist)

(25092, 239)

In [43]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

X_data = []
ood_data = []
# get the path/directory
folder_dir = "../data/ISIC2019/ISIC_2019_Training_Input/"
images = Path(folder_dir).glob("*.jpg")

for i in images:
    # image = plt.imread(i)
    image = Image.open(i).convert("RGB")
    img_resize = np.array(image.resize((224, 224), Image.Resampling.LANCZOS)).ravel()
    if i.stem in indist:
        X_data.append(img_resize)
    elif i.stem in oodist:
        ood_data.append(img_resize)
# images = list(images)
# X_array = np

In [18]:
len(X_data), len(ood_data)

(25092, 239)

In [46]:
import numpy as np

X_array = np.array(X_data)
ood_array = np.array(ood_data)

In [47]:
y_ind = np.zeros(X_array.shape[0])  # not a OOD sample
y_ood = np.ones(ood_array.shape[0])  # an OOD sample
X = np.concatenate([X_array, ood_array])
y = np.concatenate([y_ind, y_ood])
print(X.shape, y.shape)

(25331, 150528) (25331,)


#### Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf = IsolationForest(max_samples=100, random_state=0)
clf.fit(X_train)